In [1]:
import json
import requests
import pandas as pd

In [2]:
source = 'yahoo_url'
ibov = r'https://br.advfn.com/indice/ibovespa'

In [41]:
def get_url():
    with open('../ref/config.json', 'r') as f: 
        return json.load(f).get(source)

def get_quotes(symbols: list, columns: list=None):
    url = get_url().format(','.join(symbols))
    r = requests.get(url)
    if r.status_code == 200:
        body = r.json().get('quoteResponse').get('result')
        if body is not None: 
            df = pd.DataFrame(body)
            df = df if columns is None else df[columns]
            return df
        else:
            return body
    else:
        return None

def get_ibov_rics():
    ativos = pd.read_html(requests.get(ibov).text)[3][:-1]['Ativo'].to_list()
    return list(map(lambda x: x + '.SA', ativos))

def get_percent_change(change):
    values = [float(i) for i in change.split('-')]
    diff = values[1] - values[0]
    return round((diff / values[0]) * 100,2)

In [49]:
cols = ['symbol', 'longName', 'regularMarketPrice', 'regularMarketChangePercent', 'fiftyDayAverageChangePercent',
        'twoHundredDayAverageChangePercent', 'fiftyTwoWeekRange','epsTrailingTwelveMonths', 'trailingPE', 'trailingAnnualDividendYield']

adj_cols = ['fiftyDayAverageChangePercent', 'twoHundredDayAverageChangePercent', 'trailingAnnualDividendYield']

In [50]:
rics = get_ibov_rics()
df = get_quotes(rics)[cols]

In [51]:
for c in adj_cols: 
    df[c] = round(df[c] * 100,2)

    
df['12mMinMaxChange'] = df['fiftyTwoWeekRange'].apply(lambda x: get_percent_change(x))

In [59]:
df.sort_values('epsTrailingTwelveMonths', ascending=False).to_csv('ibov.csv', index=False)

In [43]:
df

,symbol,longName,regularMarketPrice,regularMarketChangePercent,fiftyDayAverageChangePercent,twoHundredDayAverageChangePercent,fiftyTwoWeekRange,epsTrailingTwelveMonths,trailingPE,trailingAnnualDividendYield,12mReturn
0,ABEV3.SA,Ambev S.A.,15.15,1.067377,16437.4300,43418.3180,10.68 - 16.91,0.717,21.129705,32688.4600,58.33
1,ASAI3.SA,Sendas Distribuidora S.A.,72.70,1.154855,10741.7920,10741.7920,67.61 - 78.5,5.210,13.953934,NaN,16.11
2,AZUL4.SA,Azul S.A.,37.37,0.429992,-87850.6200,76815.9200,11.01 - 45.64,-31.665,NaN,NaN,314.53
3,B3SA3.SA,"B3 S.A. - Brasil, Bolsa, Balcão",54.01,0.427667,-46573.9400,-55597.2680,33.16 - 69.75,2.027,26.645288,5262.1793,110.34
4,BBAS3.SA,Banco do Brasil S.A.,29.66,0.338296,-43962.5640,-111085.2060,24.0 - 40.56,4.386,6.762425,49797.0250,69.00
...,...,...,...,...,...,...,...,...,...,...,...
77,VALE3.SA,Vale S.A.,95.53,3.342705,-4867.0587,195246.6200,39.2 - 104.0,5.208,18.342934,72144.0900,165.31
78,VIVT3.SA,Telefônica Brasil S.A.,45.34,0.309733,-9779.3960,2187.3293,41.45 - 54.76,2.825,16.049557,64446.9040,32.11
79,VVAR3.SA,Via Varejo S.A.,11.97,-0.581393,-75636.0200,-259049.7000,3.96 - 22.36,0.668,17.919163,996.6778,464.65
80,WEGE3.SA,WEG S.A.,72.74,2.349793,-70652.8600,-56441.1700,31.42 - 93.86,1.115,65.237660,8372.0280,198.73


In [57]:
# get quote for single simbol
get_quotes(['CPLE6.SA']).T.to_csv('petr4.csv')